In [ ]:
import numpy as np
import pandas as pd

import scipy as sp
import scipy.stats as stats

import arviz as az
import pymc as pm

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

## Data

In [ ]:
import pickle as pk

path = 'drive/MyDrive/DATA/'
file_name = 'data_sharks.pickle'

with open(path+file_name, 'rb') as f:
    data = pk.load(f)

data.keys()

## EDA

In [ ]:
outs_len = {key:[len(i) for i in data[key]['outs']] for key in data.keys()}
inps_len = {key:[len(i) for i in data[key]['inputs']] for key in data.keys()}

In [ ]:
fit, axs = plt.subplots(1, len(outs_len.keys()), figsize=(21, 3))

for i, shark in enumerate(outs_len.keys()):
    axs[i].violinplot(outs_len[shark], showmeans=True, showmedians=True)
    axs[i].set_title(shark)

## Model

In [ ]:
outs = np.array([x for xs in list(outs_len.values()) for x in xs])
categories_o = np.array(list(data.keys()))
idx_o = pd.Categorical([key for key, val in outs_len.items() for i in val], categories_o).codes
groups_o = len(np.unique(idx_o))

In [ ]:
az.plot_forest(outs_len, kind='ridgeplot', hdi_prob=1);

In [ ]:
idx_o

In [ ]:
with pm.Model() as comparing_groups_outs:
    μ = pm.HalfNormal('μ', sigma=5, shape=groups_o)
    σ = pm.HalfNormal('σ', sigma=1, shape=groups_o)

    y = pm.Gamma('y', mu=μ[idx_o], sigma=σ[idx_o], observed=outs)

    idata_o = pm.sample()

az.plot_trace(idata_o)

In [ ]:
inputs = np.array([x for xs in list(inps_len.values()) for x in xs])
categories_i = np.array(list(data.keys()))
idx_i = pd.Categorical([key for key, val in inps_len.items() for i in val], categories_i).codes
groups_i = len(np.unique(idx_i))

In [ ]:
# ERROR. I have no idea why.
# with pm.Model() as comparing_groups_inputs:
#     μ = pm.HalfNormal('μ', sigma=5, shape=groups_i)
#     σ = pm.HalfNormal('σ', sigma=1, shape=groups_i)

#     y = pm.Gamma('y', mu=μ[idx_i], sigma=σ[idx_i], observed=inputs)

#     idata_i = pm.sample()

# az.plot_trace(idata_i)